# Word2Vec Experiments sur les Archives de Journaux

Ce notebook explore l'utilisation de Word2Vec pour analyser les textes des journaux des années 1950-1970.

## Plan
1. Chargement et préparation des données
2. Entraînement de modèles Word2Vec avec différents paramètres
3. Évaluation et comparaison des modèles
4. Analyse des relations sémantiques

## 1. Installation et importation des bibliothèques

Nous allons d'abord installer et importer les bibliothèques nécessaires.

In [ ]:
# Installation des bibliothèques nécessaires
!pip install gensim nltk tqdm
!python -m nltk.downloader punkt

In [ ]:
# Importation des bibliothèques
import os
import glob
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import logging

# Configuration du logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## 2. Chargement et préparation des données

Nous allons charger les données depuis les fichiers texte préparés. Les phrases sont déjà tokenisées dans le fichier sents.txt.

In [ ]:
# Chargement des phrases depuis sents.txt
sentences = []
with open('data/txt/sents.txt', 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        # Tokenization des mots de chaque phrase
        tokens = word_tokenize(line.strip().lower())
        if tokens:  # Ignorer les lignes vides
            sentences.append(tokens)

print(f"Nombre total de phrases chargées : {len(sentences)}")
print(f"Exemple de phrase tokenisée : {sentences[0][:30]}")  # Afficher les 30 premiers tokens d'une phrase

## 3. Entraînement des modèles Word2Vec

Nous allons entraîner plusieurs modèles Word2Vec avec différentes configurations :
- Tailles de fenêtre contextuelles : 3, 5, et 7 mots
- Fréquences minimales des mots : 3, 5, et 10 occurrences

Pour chaque combinaison, nous entraînerons un modèle et évaluerons ses performances.

In [ ]:
# Définition des paramètres à tester
window_sizes = [3, 5, 7]
min_counts = [3, 5, 10]

# Dictionnaire pour stocker les modèles
models = {}

# Entraînement des modèles avec différentes configurations
for window in window_sizes:
    for min_count in min_counts:
        print(f"\nEntraînement du modèle avec window={window}, min_count={min_count}")
        
        model = Word2Vec(sentences=sentences,
                        vector_size=100,  # Dimension des vecteurs
                        window=window,    # Taille de la fenêtre contextuelle
                        min_count=min_count,  # Fréquence minimale des mots
                        workers=4)        # Nombre de threads pour l'entraînement
        
        # Sauvegarde du modèle dans le dictionnaire
        models[f"w{window}_mc{min_count}"] = model
        
        # Affichage de quelques statistiques
        print(f"Vocabulaire : {len(model.wv.key_to_index)} mots")
        
# Sauvegarde des modèles sur disque
if not os.path.exists('models'):
    os.makedirs('models')
    
for name, model in models.items():
    model.save(f"models/word2vec_{name}.model")

## 4. Évaluation et comparaison des modèles

Nous allons maintenant évaluer les différents modèles en examinant les similarités entre les mots et en comparant les résultats obtenus avec différentes configurations.

In [ ]:
# Fonction d'évaluation pour un mot donné
def evaluate_word(model, word, topn=10):
    try:
        similar_words = model.wv.most_similar(word, topn=topn)
        print(f"\nMots les plus similaires à '{word}':")
        for w, score in similar_words:
            print(f"{w}: {score:.4f}")
    except KeyError:
        print(f"Le mot '{word}' n'est pas dans le vocabulaire du modèle")

# Liste de mots à tester
test_words = ['président', 'france', 'paris', 'guerre', 'paix', 'europe']

# Évaluation pour chaque modèle
for name, model in models.items():
    print(f"\n=== Modèle {name} ===")
    for word in test_words:
        evaluate_word(model, word)

In [ ]:
# Analyse des relations sémantiques
def test_analogy(model, word1, word2, word3, topn=5):
    try:
        results = model.wv.most_similar(positive=[word2, word3], negative=[word1], topn=topn)
        print(f"\nAnalogies pour : {word1} est à {word2} ce que {word3} est à :")
        for w, score in results:
            print(f"{w}: {score:.4f}")
    except KeyError as e:
        print(f"Erreur: un des mots n'est pas dans le vocabulaire - {e}")

# Test de quelques analogies intéressantes
analogies = [
    ('paris', 'france', 'londres'),  # Paris est à France ce que Londres est à ?
    ('homme', 'roi', 'femme'),       # Homme est à Roi ce que Femme est à ?
    ('matin', 'jour', 'soir')        # Matin est à Jour ce que Soir est à ?
]

# Test des analogies sur le modèle avec la plus grande fenêtre et min_count=5
model = models['w7_mc5']
for w1, w2, w3 in analogies:
    test_analogy(model, w1, w2, w3)